In [60]:
import os
import time
import numpy as np
import scipy.io as sio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px
import pydicom as dcm
import pandas as pd

import cv2
from skimage.morphology import binary_erosion, binary_dilation
from skimage import measure
from skimage.measure import label, regionprops, regionprops_table

from my_funcs.plot_functions import t1_t2_pixel_reader
from my_funcs.plot_functions import find_n

from my_funcs.cest_functions import dicom_data_arranger
from my_funcs.cest_functions import z_spec_rearranger
from my_funcs.cest_functions import offset_rearranger

from my_funcs.b0_mapping_functions import wassr_b0_mapping
from my_funcs.b0_mapping_functions import b0_correction

from my_funcs.cest_functions import bruker_dataset_creator


## 1.1. Enter Data: ##

In [61]:
phantom_choice = 1  # choose 1-3


In [62]:
# Root stats:
general_fn = os.path.abspath(os.curdir)
# Subject stats:
glu_mouse_fns = [os.path.join(general_fn, 'scans', 'mouse',
                                '20240128_142305_or_dino_mouse2_right_ear_middle_pierce_a_1_1')]  # Mouse
glu_mouse_fn = glu_mouse_fns[phantom_choice-1]
txt_file_name = 'labarchive_notes.txt'
fp_prtcl_name = '51_glu'


## 1.2. Create quant maps ##

In [63]:
glu_mouse_dicom_fn, glu_mouse_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_mouse_fn, txt_file_name, fp_prtcl_name)


## 1.3. Create bruker dataset and masks ##

In [64]:
quant_data_fn = os.path.join(glu_mouse_mrf_files_fn, 'quant_maps.mat')
quant_maps = sio.loadmat(quant_data_fn)
fs_array = quant_maps['fs'] * 110e3 / 3
ksw_array = quant_maps['ksw']
dp_array = quant_maps['dp']
t1w_array = quant_maps['t1w'] * 1000
t2w_array = quant_maps['t2w'] * 1000


In [65]:
acquired_data_fn = os.path.join(glu_mouse_mrf_files_fn, 'acquired_data.mat')
acquired_data = sio.loadmat(acquired_data_fn)['acquired_data']  # [31, 64, 64]
# px.imshow(acquired_data[4,:,:])


In [66]:
t1_pixels = t1_t2_pixel_reader(glu_phantom_fn=glu_mouse_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t1')
t2_pixels = t1_t2_pixel_reader(glu_phantom_fn=glu_mouse_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t2')
# px.imshow(t1_pixels, zmin=500, zmax=2500)
# px.imshow(t2_pixels, zmax=200)


## PD image for masks ##

In [67]:
pd_pixels = t1_t2_pixel_reader(glu_phantom_fn=glu_mouse_fn, txt_file_name=txt_file_name, image_idx=3, t_type='t1')

glu_phantom_txt_fn = os.path.join(glu_mouse_fn, txt_file_name)
exp_id = find_n(glu_phantom_txt_fn, 'PD_T2')
image_idx = 6  # I think they took 6
dcm_fn = os.path.join(glu_mouse_fn, f'{exp_id}/pdata/1/dicom/MRIm0{image_idx}.dcm')

dcm_data = dcm.dcmread(dcm_fn)  # Read DICOM file

# Access the rescale slope
pixel_val = dcm_data.pixel_array
rescale_slope = dcm_data.RescaleSlope
rescale_intercept = dcm_data.RescaleIntercept

if rescale_intercept != 0:
    print(f'rescale_intercept is {rescale_intercept}, should be 0!')

# Rescale
pd_data = pixel_val * rescale_slope + rescale_intercept  # rescaled
# px.imshow(pd_data)


## Brain mask ##

In [68]:
# Resize image2 to match the dimensions of image1
pd_rs_data = cv2.resize(pd_data, (t2_pixels.shape[1], t2_pixels.shape[0]))
pd_rs_data_8 = pd_rs_data.astype('uint8')
_, image_binary = cv2.threshold(pd_rs_data_8, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Specify the minimum number of pixels for separating regions
min_pixels = 10

# Create circular structuring element
radius = int(min_pixels / 2)
circular_footprint = np.zeros((min_pixels, min_pixels), dtype=bool)
y, x = np.ogrid[:min_pixels, :min_pixels]
mask = (x - radius) ** 2 + (y - radius) ** 2 <= radius ** 2
circular_footprint[mask] = 1

# Perform erosion and dilation using circular structuring element
binary_separated_1 = binary_dilation(image_binary != 0, footprint=circular_footprint)
binary_separated_2 = binary_erosion(binary_separated_1, footprint=circular_footprint)

brain_mask = label(~binary_separated_2)

# px.imshow(brain_mask*pd_rs_data_8)
# px.imshow(pd_rs_data)


In [69]:
import matplotlib.pyplot as plt

# regionprops(label_image)[0].bbox  # 20
centroid = regionprops(brain_mask)[0].centroid  # 20
orientation = regionprops(brain_mask)[0].orientation + 0.01  # again manual

# regionprops(label_image)[0].
# Plot the orientation line
x0, y0 = centroid[1], centroid[0]
x1 = x0 - np.cos(orientation) * 20
y1 = y0 + np.sin(orientation) * 20
x2 = x0 + np.cos(orientation) * 20
y2 = y0 - np.sin(orientation) * 20
# plt.imshow(brain_mask*pd_rs_data_8)
# plt.plot(centroid[1], centroid[0], 'ro', label='Centroid')
# plt.plot([x1, x2], [y1, y2], 'g-', linewidth=2, label='Orientation')
# plt.show()


## Tumor mask ##

In [70]:
brain_im = brain_mask*pd_rs_data
brain_im_th = (brain_im>56)  # I chose manually
label_brain_im = label(brain_im_th)
px.imshow(label_brain_im)

# Find the regions that correspond to the circles
regions = regionprops(label_brain_im)
region = regions[1]  # I chose manually
tumor_mask = np.zeros_like(label_brain_im)
rr, cc = region.coords[:, 0], region.coords[:, 1]
tumor_mask[rr, cc] = 1

# # Specify the minimum number of pixels for separating regions (temp???)
# min_pixels = 2
# # Create circular structuring element
# radius = int(min_pixels / 2)
# circular_footprint = np.zeros((min_pixels, min_pixels), dtype=bool)
# y, x = np.ogrid[:min_pixels, :min_pixels]
# mask = (x - radius) ** 2 + (y - radius) ** 2 <= radius ** 2
# circular_footprint[mask] = 1
# tumor_mask = binary_erosion(tumor_mask != 0, footprint=circular_footprint)  # mask dilation

# px.imshow(tumor_mask*brain_im)


In [71]:
def mask_mirrorer(mask, center_line):
    (x0, y0), (x1, y1) = center_line
    height, width = mask.shape
    mirrored_mask = np.zeros_like(mask)

    # Find the indices where the mask is 1
    indices = np.argwhere(mask == 1)

    # Calculate the vector from (x0, y0) to (x1, y1)
    dx = x1 - x0
    dy = y1 - y0
    
    # Calculate the length of the vector
    length = np.sqrt(dx ** 2 + dy ** 2)
    
    # Normalize the vector
    dx /= length
    dy /= length
    
    # Calculate the normal vector (perpendicular to the centerline)
    normal_x = -dy
    normal_y = dx

    # Iterate through each point in the original mask
    for y, x in indices:
        # Calculate the distance from the point (x, y) to the centerline
        distance = (x - x0) * normal_x + (y - y0) * normal_y
        
        # Calculate the mirrored coordinates
        x_mirror = int(x - 2 * distance * normal_x)
        y_mirror = int(y - 2 * distance * normal_y)

        # Check if the mirrored coordinates are within the mask boundaries
        if 0 <= x_mirror < width and 0 <= y_mirror < height:
            # Mark the mirrored pixel in the mirrored mask as 1
            mirrored_mask[y_mirror, x_mirror] = 1

    return mirrored_mask

center_line = (x1, y1), (x2, y2)
mirror_mask = mask_mirrorer(tumor_mask, center_line)


In [72]:
def contour_finder(cur_mask):
    contour = measure.find_contours(image=cur_mask, level=0.5)[0]
    shift = 0.5
    contour[:, 1] += shift
    contour[:, 0] += shift
    
    # Find and add additional points to improve the contour plot
    points = []
    for idx0 in range(len(contour)):
        pos0 = contour[idx0]
        idx1 = idx0 + 1
        if len(contour) > idx1:
            pos1 = contour[idx1]
            y1, x1 = tuple(pos1)
            y0, x0 = tuple(pos0)
            if y1 - y0 and x1 - x0:
                point = np.array([y0, x1])
                if y0 % 1:
                    point = np.array([y1, x0])
                points.append((idx0, point))
    points.reverse()
    for idx0, point in points:
        contour = np.insert(contour, idx0 + 1, point, axis=0)
    
    # Deshift the contour before returning
    contour[:, 1] -= shift
    contour[:, 0] -= shift
        
    return contour

tumor_cntr = contour_finder(tumor_mask)  # not accurate needs fixing!
mirror_cntr = contour_finder(mirror_mask)


In [73]:
# Create subplots with 1 row and 3 columns, increased horizontal spacing
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.13, subplot_titles=['Real t1 [ms]', 'Real t2 [ms]'])

# Add heatmaps for the three arrays
heatmap_t1 = go.Heatmap(z=brain_mask * t1_pixels, colorscale='Viridis', coloraxis="coloraxis1")
heatmap_t2 = go.Heatmap(z=brain_mask * t2_pixels, colorscale='Viridis', coloraxis="coloraxis2")
fig.add_trace(heatmap_t1, row=1, col=1)
fig.add_trace(heatmap_t2, row=1, col=2)

# Create a scatter plot for the contour using Plotly Express
fig.add_trace(go.Scatter(x=tumor_cntr[:, 1], y=tumor_cntr[:, 0], line=dict(color="red", width=1.5)), row=1, col=1)
fig.add_trace(go.Scatter(x=mirror_cntr[:, 1], y=mirror_cntr[:, 0], line=dict(color="red", width=1.5)), row=1, col=1)
fig.add_trace(go.Scatter(x=tumor_cntr[:, 1], y=tumor_cntr[:, 0], line=dict(color="red", width=1.5)), row=1, col=2)
fig.add_trace(go.Scatter(x=mirror_cntr[:, 1], y=mirror_cntr[:, 0], line=dict(color="red", width=1.5)), row=1, col=2)

[[t1_tumor_mean]], [[t1_tumor_std]] = cv2.meanStdDev(t1_pixels, mask=tumor_mask.astype(np.uint8))
[[t2_tumor_mean]], [[t2_tumor_std]] = cv2.meanStdDev(t2_pixels, mask=tumor_mask.astype(np.uint8))
[[t1_mirror_mean]], [[t1_mirror_std]] = cv2.meanStdDev(t1_pixels, mask=mirror_mask.astype(np.uint8))
[[t2_mirror_mean]], [[t2_mirror_std]] = cv2.meanStdDev(t2_pixels, mask=mirror_mask.astype(np.uint8))

fig.update_layout(
        template='plotly_dark',  # Set the theme to plotly dark
        title_text=f'Mouse Brain',
        showlegend=False,  # Hide legend
        height=345,
        width=680,  # Set a width based on your preference
        margin=dict(l=10, r=0, t=60, b=20),  # Adjust top and bottom margins
        title=dict(x=0.02, y=0.97)  # Adjust the title position
    )

# Add individual titles and separate colorbars
for i, title in enumerate(['Real t1 [ms]', 'Real t2 [ms]'], start=1):
    fig.update_xaxes(row=1, col=i, showgrid=False, showticklabels=False)
    fig.update_yaxes(showgrid=False, row=1, col=i, showticklabels=False, autorange='reversed')  # Reverse the y-axis

# Manually add separate colorbars
colorbar_t1 = {'colorscale': 'Viridis'}
colorbar_t2 = {'colorscale': 'Plasma'}

fig.update_layout(
    coloraxis1=colorbar_t1,
    coloraxis2=colorbar_t2,
    coloraxis_colorbar=dict(x=0.43, y=0.5),
    coloraxis2_colorbar=dict(x=0.995, y=0.5),
)
# Show the plot
fig.show()

# Define the data
data = {
    'tumor': [f'{round(t1_tumor_mean, 2)} ± {round(t1_tumor_std, 2)}', f'{round(t2_tumor_mean, 2)} ± {round(t2_tumor_std, 2)}'],
    'mirror': [f'{round(t1_mirror_mean, 2)} ± {round(t1_mirror_std, 2)}', f'{round(t2_mirror_mean, 2)} ± {round(t2_mirror_std, 2)}']
}

df = pd.DataFrame(data, index=['t1', 't2'])
display(df)


,tumor,mirror
t1,1645.64 ± 129.78,1402.01 ± 55.83
t2,67.41 ± 13.87,43.72 ± 1.5


In [74]:
# Create subplots with 1 row and 3 columns, increased horizontal spacing
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.13, subplot_titles=['fs [mM]', 'ksw [Hz]'])

# Add heatmaps for the three arrays
heatmap_t1 = go.Heatmap(z=brain_mask * fs_array, colorscale='Viridis', coloraxis="coloraxis1")
heatmap_t2 = go.Heatmap(z=brain_mask * ksw_array, colorscale='Viridis', coloraxis="coloraxis2")
fig.add_trace(heatmap_t1, row=1, col=1)
fig.add_trace(heatmap_t2, row=1, col=2)

# Create a scatter plot for the contour using Plotly Express
fig.add_trace(go.Scatter(x=tumor_cntr[:, 1], y=tumor_cntr[:, 0], line=dict(color="red", width=1.5)), row=1, col=1)
fig.add_trace(go.Scatter(x=mirror_cntr[:, 1], y=mirror_cntr[:, 0], line=dict(color="red", width=1.5)), row=1, col=1)
fig.add_trace(go.Scatter(x=tumor_cntr[:, 1], y=tumor_cntr[:, 0], line=dict(color="red", width=1.5)), row=1, col=2)
fig.add_trace(go.Scatter(x=mirror_cntr[:, 1], y=mirror_cntr[:, 0], line=dict(color="red", width=1.5)), row=1, col=2)

[[fs_tumor_mean]], [[fs_tumor_std]] = cv2.meanStdDev(fs_array, mask=tumor_mask.astype(np.uint8))
[[ksw_tumor_mean]], [[ksw_tumor_std]] = cv2.meanStdDev(ksw_array, mask=tumor_mask.astype(np.uint8))
[[fs_mirror_mean]], [[fs_mirror_std]] = cv2.meanStdDev(fs_array, mask=mirror_mask.astype(np.uint8))
[[ksw_mirror_mean]], [[ksw_mirror_std]] = cv2.meanStdDev(ksw_array, mask=mirror_mask.astype(np.uint8))

fig.update_layout(
        template='plotly_dark',  # Set the theme to plotly dark
        title_text=f'Mouse Brain',
        showlegend=False,  # Hide legend
        height=345,
        width=680,  # Set a width based on your preference
        margin=dict(l=10, r=0, t=60, b=20),  # Adjust top and bottom margins
        title=dict(x=0.02, y=0.97)  # Adjust the title position
    )

# Add individual titles and separate colorbars
for i, title in enumerate(['fs [mM]', 'ksw [Hz]'], start=1):
    fig.update_xaxes(row=1, col=i, showgrid=False, showticklabels=False)
    fig.update_yaxes(showgrid=False, row=1, col=i, showticklabels=False, autorange='reversed')  # Reverse the y-axis

# Manually add separate colorbars
colorbar_t1 = {'colorscale': 'Viridis'}
colorbar_t2 = {'colorscale': 'Plasma'}

fig.update_layout(
    coloraxis1=colorbar_t1,
    coloraxis2=colorbar_t2,
    coloraxis_colorbar=dict(x=0.43, y=0.5),
    coloraxis2_colorbar=dict(x=0.995, y=0.5),
)
# Show the plot
fig.show()

# Define the data
data = {
    'tumor': [f'{round(fs_tumor_mean, 2)} ± {round(fs_tumor_std, 2)}', f'{round(ksw_tumor_mean, 2)} ± {round(ksw_tumor_std, 2)}'],
    'mirror': [f'{round(fs_mirror_mean, 2)} ± {round(fs_mirror_std, 2)}', f'{round(ksw_mirror_mean, 2)} ± {round(ksw_mirror_std, 2)}']
}

df = pd.DataFrame(data, index=['fs', 'ksw'])
display(df)


,tumor,mirror
fs,9.94 ± 5.03,11.28 ± 4.63
ksw,6250.0 ± 1204.74,6366.67 ± 1286.25


In [75]:
from my_funcs.plot_functions import plot_norm_sig
from numpy import linalg as la

def norm_roi(roi_mask, quant_data_fn, mrf_files_fn):
    """
    Create normalized mrf signals
    :param: roi_mask:
    :param: quant_data_fn:
    :param: mrf_files_fn:
    :return norm_vial_real_sig: real signal from acquit
    :return norm_vial_match_sig: dict match signal
    """
    quant_maps = sio.loadmat(quant_data_fn)

    acquired_data_fn = os.path.join(mrf_files_fn, 'acquired_data.mat')
    acquired_data = sio.loadmat(acquired_data_fn)['acquired_data']

    dict_fn = os.path.join(mrf_files_fn, 'dict.mat')
    synt_dict = sio.loadmat(dict_fn)
    synt_sig = np.transpose(synt_dict['sig'])   # e.g. 30 x 369,000
    synt_fs = synt_dict['fs_0']
    synt_ksw = synt_dict['ksw_0']
    synt_t1w = synt_dict['t1w']
    synt_t2w = synt_dict['t2w']

    acquired_data = acquired_data[1:, :]
    if synt_sig.shape[0] == 31:
        synt_sig = synt_sig[1:, :]

    x_locs, y_locs = np.where(roi_mask)

    m_ids_pixel = quant_maps['match_id'][x_locs, y_locs].astype('int')
    # exp_id_pixel = np.where((synt_fs == expected_f[vial_i]) & (synt_ksw == expected_k[vial_i]) & (synt_t1w == expected_t1[vial_i]) & (synt_t2w == expected_t2[vial_i]))[1][0]
    real_sig = acquired_data[:, x_locs, y_locs]
    n_real_sig = real_sig / la.norm(real_sig, axis=0)
    n_real_sig_mean = np.mean(n_real_sig, axis=1)
    n_real_sig_std = np.std(n_real_sig, axis=1)

    match_sig = synt_sig[:, m_ids_pixel]
    n_match_sig = match_sig / la.norm(match_sig, axis=0)
    n_match_sig_mean = np.mean(n_match_sig, axis=1)
    n_match_sig_std = np.std(n_match_sig, axis=1)

    return (n_real_sig_mean, n_match_sig_mean), (n_real_sig_std, n_match_sig_std), (n_real_sig, n_match_sig)


In [76]:
import plotly.graph_objects as go

mean_sigs, std_sigs, mat_sigs = norm_roi(tumor_mask, quant_data_fn, glu_mouse_mrf_files_fn)  # real, matched, expected

x_axis = np.arange(0, len(mean_sigs[0]))
fig = go.Figure(data=go.Scatter(
        x=x_axis,
        y=mean_sigs[0],
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=std_sigs[0],
            visible=True)
    ))
fig.show()


In [77]:
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.07, horizontal_spacing=0.01)

# Colors for the scatter plots
color_1 = '#1f77b4'
color_2 = '#ff7f0e'
color_3 = '#2ca02c'

for roi_i, roi_mask in enumerate([tumor_mask, mirror_mask]):
    mean_sigs, std_sigs, _ = norm_roi(roi_mask, quant_data_fn, glu_mouse_mrf_files_fn)  # real, matched, expected
    
    x_axis = np.arange(0, len(mean_sigs[0]))
    # Add heatmaps for the three arrays
    scatter_real = go.Scatter(x=x_axis, y=mean_sigs[0], mode='lines', name='Real Signal', line=dict(color=color_1),
                          error_y=dict(type='data', # value of error bar given in data coordinates
                                       array=std_sigs[0],
                                        visible=True))
    scatter_match = go.Scatter(x=x_axis, y=mean_sigs[1], mode='lines', name='Matched Signal', line=dict(color=color_2),
                              error_y=dict(type='data', # value of error bar given in data coordinates
                                           array=std_sigs[1],
                                           visible=True))
    
    # Add traces to the first subplot
    fig.add_trace(scatter_real, row=roi_i+1, col=1)
    fig.add_trace(scatter_match, row=roi_i+1, col=1)
    
    fig.update_yaxes(title_text=f'Normalized signal', row=roi_i+1, col=1)

# Set layout for better visualization
fig.update_layout(
    template='plotly_white',  # Set the theme to plotly white
    title_text=f'{fp_prtcl_name}, signal close look',
    showlegend=True,  # Hide legend
    legend=dict(x=0.8, y=1.05),
    height=600,
    width=700  # Set a width based on your preference
)

labels_to_show_in_legend = ['Real Signal', 'Matched Signal']

# only show 3 first traces in legend!
for trace_i, trace in enumerate(fig['data']):
    if trace_i > 1:
        trace['showlegend'] = False

# Adjust margin to reduce whitespace
fig.update_layout(margin=dict(l=0, r=5, t=60, b=0))
# Set y-axis ticks to values in the range [0, 1] with a step of 0.2
# fig.update_yaxes(tickmode='linear', tick0=0, dtick=0.1, range=[0.05, 0.4])

# Show the plot
fig.show()


## z-spectrum ##

In [78]:
cest_prtcl_names = ['cest_EPI_0p7uT', 'cest_EPI_2uT']
B_name = ['0p7uT', '2uT']

gyro_ratio_hz = 42.5764  # for H [Hz/uT]
b0 = 7


In [79]:
def z_spec_calculator(b1_str, roi_mask, b0_map):
    gyro_ratio_hz = 42.5764  # for H [Hz/uT]
    b0 = 7
    # z-spec
    glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_mouse_fn, txt_file_name, f'cest_EPI_{b1_str}uT')  # (58/52, 64, 64) [M0,7,-7,6.75,-6.75,...,0.25,-0.25,0]
    cest_data = dicom_data_arranger(bruker_dataset, glu_phantom_dicom_fn)
    M0_cest, arr_z_spec = z_spec_rearranger(cest_data)
    z_spec_norm = np.divide(arr_z_spec, np.where(M0_cest == 0, 1e-8, M0_cest))  # (51/57, 64, 64) full_mask
    
    # offset vector
    offset_hz = offset_rearranger(bruker_dataset['SatFreqList'].value)
    offset_ppm = offset_hz / (gyro_ratio_hz * b0)

    b0_cor_zspec = b0_correction(b0_map, z_spec_norm, offset_hz) # have not checked!
    
    cha_n = z_spec_norm.shape[0]
    mean_z_uc = np.zeros([cha_n])
    mean_z = np.zeros([cha_n])
    for c_i in range(cha_n):
        [[mean_z_uc[c_i]]], _ = cv2.meanStdDev(z_spec_norm[c_i, :, :], mask=roi_mask.astype(np.uint8))  # before b0 correction
        [[mean_z[c_i]]], _ = cv2.meanStdDev(b0_cor_zspec[c_i, :, :], mask=roi_mask.astype(np.uint8))
        
    # MTR asym calculation
    mid_i = int(cha_n/2 - 1)
    last_i = int(cha_n-1)
    
    positives = mean_z_uc[0:(mid_i+2)]  # positives to 0 (including)
    negatives = mean_z_uc[last_i:mid_i:-1]  # negatives to 0 (including)
    MTR_asym_uc = negatives-positives
    
    positives = mean_z[0:(mid_i+2)]  # positives to 0 (including)
    negatives = mean_z[last_i:mid_i:-1]  # negatives to 0 (including)
    MTR_asym = negatives-positives
    
    ppm_lim = round(offset_ppm[0])
    ppm_asym = np.linspace(ppm_lim, 0, num=(mid_i+2))     

    ppm = np.linspace(ppm_lim, -ppm_lim, num=cha_n)
        
    return ppm, ppm_asym, mean_z, MTR_asym, mean_z_uc, MTR_asym_uc


In [80]:
# WASSR image
wassr_dicom_fn, wassr_mrf_files_fn, wassr_bruker_dataset = bruker_dataset_creator(glu_mouse_fn, txt_file_name, 'WASSR')
wassr_data = dicom_data_arranger(wassr_bruker_dataset, wassr_dicom_fn)
M0_wassr, arr_wassr_spec = z_spec_rearranger(wassr_data)  # (21, 64, 64)

wassr_norm = np.divide(arr_wassr_spec, np.where(M0_wassr == 0, 1e-8, M0_wassr))  # (22, 64, 64) full_mask
offset_hz = offset_rearranger(wassr_bruker_dataset['SatFreqList'].value)
offset_ppm = offset_hz / (gyro_ratio_hz * b0)
b0_map = wassr_b0_mapping(wassr_norm, brain_mask, w_x=offset_ppm, MainFieldMHz=gyro_ratio_hz*b0)


D:\OneDrive - mail.tau.ac.il\master\09-code\glu-cest\my_funcs\b0_mapping_functions.py:17: RuntimeWarning:

divide by zero encountered in true_divide



WASSR B0 mapping took 1.175 seconds


In [81]:
ppm, ppm_asym, mean_z, MTR_asym, mean_z_uc, MTR_asym_uc = z_spec_calculator('0p7', tumor_mask, b0_map)


In [84]:
B1s_l =['0p7', '2']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

fig_z = make_subplots(rows=1, cols=len(cest_prtcl_names),
                    subplot_titles=B1s_l, vertical_spacing = 0.07, horizontal_spacing=0.01)
fig_mtr = make_subplots(rows=1, cols=len(cest_prtcl_names),
                      subplot_titles=B1s_l, vertical_spacing = 0.07, horizontal_spacing=0.05)

# Iterate over each channel
for B1_i, B1_s in enumerate(B1s_l):
    col_idx = B1_i + 1
    for mask_i, (mask_name, mask) in enumerate(zip(['Tumor', 'Mirror'], [tumor_mask, mirror_mask])):
        ppm, ppm_asym, mean_z, MTR_asym, mean_z_uc, MTR_asym_uc = z_spec_calculator(B1_s, mask, b0_map)
        fig_z.add_trace(go.Scatter(x=ppm, y=mean_z, line=dict(color=colors[mask_i]), name=f'{mask_name}', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)
        # fig_z.add_trace(go.Scatter(x=ppm, y=mean_z_uc, line=dict(color=colors[-1]), name=f'{mask_name}_uc', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)
     
        fig_z.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[mask_i]), name=f'{mask_name}', legendgroup=f'{1}_{col_idx}_2'), row=1, col=col_idx)
        fig_z.update_xaxes(title_text='ppm', row=1, col=col_idx)
        
        fig_mtr.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[mask_i]), name=f'{mask_name}', legendgroup=f'{1}_{col_idx}_2'), row=1, col=col_idx)
        fig_mtr.update_xaxes(title_text='ppm', row=1, col=col_idx)
        
fig_z.update_yaxes(title_text='$M_{sat}/M_0$', row=1, col=1, title_standoff=2)
fig_mtr.update_yaxes(title_text='MTR-asym', row=1, col=1)

# Update layout
fig_z.update_layout(template='plotly_white',  # Set the theme to plotly white
                  title_text=f'Z-spectrum',
                  height=250, width=1000,
                  title=dict(x=0.02, y=0.97),
                  margin=dict(l=45, r=0, t=50, b=0)
                  )  # Adjust the title position

fig_mtr.update_layout(template='plotly_white',  # Set the theme to plotly white
              title_text=f'MTR-asym',
              height=250, width=800,
              title=dict(x=0.02, y=0.97),
              margin=dict(l=0, r=0, t=60, b=0)
                    )  # Adjust the title position

# Set axes for z-spectrum
fig_z.update_xaxes(autorange='reversed',tickmode='linear', tick0=0, dtick=1)
fig_z.update_yaxes(tickmode='linear', tick0=0, dtick=0.2, range=[-0.1, 1])

# # Set axes for MTR-asym
fig_mtr.update_xaxes(autorange='reversed', dtick=1)
# fig_2.update_xaxes(tickvals=list(range(5)), ticktext=list(map(str, range(4, -1, -1))))
fig_mtr.update_yaxes(tickmode='linear', tick0=0, dtick=0.05, range=[-0.03, 0.25], tickformat='.0%')    

# Show only specific legend groups
for trace in fig_z.data:
    trace.showlegend = (trace.legendgroup == '1_1_1')
    
# Show only specific legend groups
for trace in fig_mtr.data:
    trace.showlegend = (trace.legendgroup == '1_1_2')

fig_z.show()
fig_mtr.show()

# pio.write_image(fig, f'images/z_spec/zspec_phantom_{phantom_i+1}.jpeg')
# pio.write_image(fig_2, f'images/z_spec/mtr_phantom_{phantom_i+1}.jpeg')
